In [ ]:
!pip install --upgrade openai

In [ ]:
import os
OPENAI_API_KEY = "Private OPENAI Key"
from openai import OpenAI


client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
def upload_file(file_path):

	# Upload a file with an "assistants" purpose
	file_to_upload = client.files.create(
  	file=open(file_path, "rb"),
  	purpose='assistants'
	)
     return file_to_upload

In [ ]:
file_path = "./data/xyz_file_path.pdf"   #Insert the path of file that you want to upload
file_to_upload = upload_file(file_path)

In [ ]:
def create_assistant(assistant_name ,my_instruction ,uploaded_file ,model="gpt-4-1106-preview"):
  #Creating Assistant
	my_assistant = client.beta.assistants.create(
	name = assistant_name,
	instructions = my_instruction,
	model="gpt-4-1106-preview",
	tools=[{"type": "retrieval"}],
	file_ids=[uploaded_file.id]
	)

	return my_assistant

In [ ]:
prompt_template = """
You are Generative AI Assistant. Create a personalized study plan based on the following data:

1. Subjects and Academic Performance:
   - Subjects and grades/performance levels

2. Preferred Learning Styles:
   - Visual, Auditory, Kinesthetic

3. Extracurricular Activities:
   - List of activities

4. Personal Objectives or Challenges:
   - Exams, learning difficulties, personal goals

Include in the study plan:

- Weekly study schedule
- Study techniques and resources for the learning style
- Balance strategies for academics and activities
- Action steps for objectives and challenges
- Motivational tips

Ensure the plan is realistic, achievable, and tailored to the student's unique needs. Present it clearly and structured.
"""

In [ ]:
assistant_name="Generative AI Assistant "

my_assistant = create_assistant(assistant_name, prompt_template, file_to_upload)

In [ ]:
def initiate_interaction(user_message, uploaded_file):
    """The code defines a function initiate_interaction that takes user_message and uploaded_file as parameters. It performs the following steps:
        Creates a new thread (my_thread) using client.beta.threads.create().
        Sends a message in the thread using client.beta.threads.messages.create() with the thread ID, sender's role, message content, and uploaded file ID.
        Returns my_thread."""
    my_thread = client.beta.threads.create()
    message = client.beta.threads.messages.create(thread_id=my_thread.id,
                                              	role="user",
                                              	content=user_message,
                                              	file_ids=[uploaded_file.id]
	)
    return my_thread

In [ ]:
user_message = """Generate a concise summary of the book "Crime and Punishment" limited to 20 pages."""
my_thread = initiate_interaction(user_message, file_to_upload)

In [ ]:
def trigger_assistant():
	run = client.beta.threads.runs.create(
  	thread_id = my_thread.id,
  	assistant_id = my_assistant.id,
	)
trigger_assistant()

In [ ]:
messages = client.beta.threads.messages.list(
  thread_id = my_thread.id
)

In [ ]:
response = messages.data[0].content[0].text.value
response

In [ ]:
!pip install fpdf

In [ ]:
from fpdf import FPDF

def save_text_to_pdf(text, file_path):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    pdf.multi_cell(0, 10, text)

    pdf.output(file_path)


file_path = "output.pdf"
save_text_to_pdf(response, file_path)

print(f"PDF saved to {file_path}")

In [ ]:
"""NOTE 
We offer two options for providing a file to the assistant. The first option is to provide the file directly during the creation of the assistant. The second option is to provide the file in a thread when the user submits a query.

The key outcomes are:

Providing the file ID during assistant creation: This approach generates an assistant specific to that particular file. However, it requires creating multiple assistants for different files, leading to higher costs and limitations.
Providing the assistant ID and file ID in a thread: This method is more efficient and cost-effective, as a single assistant can handle the entire project.

Choosing the second option saves costs and allows one assistant to manage the whole project effectively.
"""